In [ ]:
# MLP - with using paper Bengio et al.2003

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [3]:
# read in all the words
words = open('cleaned_names.txt', 'r').read().splitlines()
words[:8]

['aaban', 'aabid', 'aabidah', 'aabir', 'aabriella', 'aada', 'aadam', 'aadarsh']

In [4]:
len(words)

29681

In [5]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(set(''.join(words)))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [10]:
# build the dataset

block_size = 3 # context length: how many characters to use to predict the next one?
X, Y = [], []
for w in words[:5]:

  print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join([itos[i] for i in context]), '---->', itos[ix])
    context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

aaban
... ----> a
..a ----> a
.aa ----> b
aab ----> a
aba ----> n
ban ----> .
aabid
... ----> a
..a ----> a
.aa ----> b
aab ----> i
abi ----> d
bid ----> .
aabidah
... ----> a
..a ----> a
.aa ----> b
aab ----> i
abi ----> d
bid ----> a
ida ----> h
dah ----> .
aabir
... ----> a
..a ----> a
.aa ----> b
aab ----> i
abi ----> r
bir ----> .
aabriella
... ----> a
..a ----> a
.aa ----> b
aab ----> r
abr ----> i
bri ----> e
rie ----> l
iel ----> l
ell ----> a
lla ----> .


In [23]:
X.shape, X.dtype, Y.shape, Y.dtype 

(torch.Size([36, 3]), torch.int64, torch.Size([36]), torch.int64)

In [24]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1,  1],
        [ 1,  1,  2],
        [ 1,  2,  1],
        [ 2,  1, 14],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1,  1],
        [ 1,  1,  2],
        [ 1,  2,  9],
        [ 2,  9,  4],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1,  1],
        [ 1,  1,  2],
        [ 1,  2,  9],
        [ 2,  9,  4],
        [ 9,  4,  1],
        [ 4,  1,  8],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1,  1],
        [ 1,  1,  2],
        [ 1,  2,  9],
        [ 2,  9, 18],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1,  1],
        [ 1,  1,  2],
        [ 1,  2, 18],
        [ 2, 18,  9],
        [18,  9,  5],
        [ 9,  5, 12],
        [ 5, 12, 12],
        [12, 12,  1]])

In [25]:
Y # labels

tensor([ 1,  1,  2,  1, 14,  0,  1,  1,  2,  9,  4,  0,  1,  1,  2,  9,  4,  1,
         8,  0,  1,  1,  2,  9, 18,  0,  1,  1,  2, 18,  9,  5, 12, 12,  1,  0])

In [30]:
C = torch.randn(27, 2) # each 27 characters is represented by a 2D embedding

In [31]:
C

tensor([[ 1.4496, -0.3366],
        [-1.4357, -0.5382],
        [-0.8678, -0.1583],
        [-0.2588,  0.2427],
        [ 0.0719, -2.0738],
        [ 0.3601,  1.0921],
        [-0.5043,  2.0323],
        [ 0.9437, -1.0060],
        [ 1.0428, -1.2123],
        [-0.0516,  0.4853],
        [-2.2135, -1.1812],
        [-0.0229,  1.0469],
        [-0.1798, -0.0718],
        [-1.0753, -1.1747],
        [ 0.6411, -0.2531],
        [ 2.9768, -1.4304],
        [-0.1134,  2.2262],
        [ 0.3963,  0.6176],
        [ 0.1623, -0.3596],
        [-0.7004,  0.9535],
        [ 0.4353,  0.7417],
        [ 2.1362, -0.4532],
        [-0.3046,  0.3165],
        [-1.5889, -0.1223],
        [-0.4992,  1.7306],
        [ 0.8544,  0.1682],
        [-0.7112,  0.3572]])

In [33]:
C[5]

tensor([0.3601, 1.0921])

In [34]:
F.one_hot(torch.tensor(5), num_classes=27).float() @ C # one hot vector

tensor([0.3601, 1.0921])

In [35]:
C[[5,6,7]] # indexing with list

tensor([[ 0.3601,  1.0921],
        [-0.5043,  2.0323],
        [ 0.9437, -1.0060]])

In [36]:
C[torch.tensor([5,6,7])] # indexing with tensor

tensor([[ 0.3601,  1.0921],
        [-0.5043,  2.0323],
        [ 0.9437, -1.0060]])

In [39]:
C[X].shape

torch.Size([36, 3, 2])

In [40]:
X[13,2] # 13th example, 2nd character

tensor(1)

In [41]:
C[X[13,2]] # embedding of the 13th example, 2nd character

tensor([-1.4357, -0.5382])

In [42]:
C[1]

tensor([-1.4357, -0.5382])

In [43]:
emb = C[X]
emb.shape

torch.Size([36, 3, 2])